Modeling 
This notebook contains our machine learning algorithm (Naive Bayes: partially implemented, logistic regression: not implemented yet, ...). It builds on the preprocessing steps (bag-of-words and tf-idf). 

In [7]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import StandardScaler

Naive Bayes algorithm

In [9]:
# Load the TF-IDF scores
tfidf_path = "data/intermediate/tfidf_scores.csv"
tfidf_df = pd.read_csv(tfidf_path)

# Generate labels heuristically (Example: Sentiment Lexicon or Clustering)
# Placeholder: Random binary labels for now
y = np.random.choice([0, 1], size=len(tfidf_df))

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_df, y, test_size=0.2, random_state=42)

# Train Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

# Predict on test set
y_pred = nb_classifier.predict(X_test)

# Evaluate performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print(classification_report(y_test, y_pred))

# Cross-validation
cv_scores = cross_val_score(nb_classifier, tfidf_df, y, cv=5)
print(f"Cross-validation accuracy: {cv_scores.mean():.4f}")

FileNotFoundError: [Errno 2] No such file or directory: 'data/intermediate/tfidf_scores.csv'

Multinomial Logistic Regression algorithm

In [8]:
# Load dataset
file_path = "D:/TechLab Project/techlabs-data-science-yelp/data/intermediate/bow_vectors.csv"
df = pd.read_csv(file_path)

# Assume the first column is the text reviews (not used in ML model) and the second column is "sentiment label"
X = df.iloc[:, 2:]  # Features (numerical representation of reviews)
y = df.iloc[:, 1]   # Target (sentiment label)

# Split into train-test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Train Multinomial Logistic Regression
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=500)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Classification report
print("Classification Report:\n", classification_report(y_test, y_pred))


c:\Users\tuanl\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.79      0.82       213
           1       0.91      0.96      0.94       647
           2       0.34      0.24      0.28        58

    accuracy                           0.87       918
   macro avg       0.70      0.66      0.68       918
weighted avg       0.86      0.87      0.87       918

